In [1]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
img_url = 'download (1).jpeg'
raw_image = Image.open(img_url).convert('RGB')
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# conditional image captioning
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))
# >>> a photography of a woman and her dog

# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a photography of a group of friends taking a self self self self self self self self self self
a group of people standing in a street


In [41]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNetFromDarknet('darknet/cfg/yolov4.cfg', 'yolov4.weights')
# Specify configuration and weights files for the model
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load names of YOLO classes
classes = []
with open("darknet/data/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Function to get the output layer names
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers
# Function to perform object detection and print results
def detect_objects(image):
    height, width, _ = image.shape

    # Prepare the image for YOLO model
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Run forward pass through the network
    outputs = net.forward(get_output_layers(net))

    detected_objects = []
    detected_faces = []
    # Lists to store bounding boxes, confidences, and class IDs
    boxes = []
    confidences = []
    class_ids = []

    # Process detections
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                print(x,y,w,h)
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression (NMS)
    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.2)
    detected_faces = []
    for i in indices:
        # i = i[0]
    
        box = boxes[i]
        x, y, w, h = box
        class_id = class_ids[i]
        x = max(0,x)
        y = max(0,y)
        print(classes[class_id])
        # Filter for "person" class (adjust for other relevant classes)
        if classes[class_id] == 'person':
            # Extract face region from the person bounding box (example: just the upper half)
            face_region = image[y:y+h//2, x:x+w]

            # Append detected face region to the list
            detected_faces.append(face_region)
        
            # Draw rectangle around the detected person (optional)
            # cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            # cv2.putText(image, classes[class_id], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image, detected_faces
image_path = 'now3.jpg'  # Replace with your image path
image = cv2.imread(image_path)
detected_image , detected_faces = detect_objects(image)

i = 0

for face in (detected_faces):
    try:
        cv2.imwrite(f'face_{i}.jpg', face)
    except:
        print("Error")
    i = i+1
# Display the detected image
# cv2.imshow('YOLOv4 Object Detection', detected_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


291 200 160 285
-4 201 307 285
-4 201 307 284
-5 201 311 285
283 206 168 277
280 206 171 278


AttributeError: module 'tensorflow._api.v2.compat.v1' has no attribute 'estimator'